In [87]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [88]:
import pandas as pd
from pulp import *

# 读取数据
file_path = r"C:\Users\zhangyutang\Desktop\附件3_处理_final_modified.xlsx"
df = pd.read_excel(file_path)

# 清理列名，去掉换行符和空格
df.columns = df.columns.str.replace("\n", "").str.strip()

# 重命名字段为英文，便于处理
df.rename(columns={"餐次": "meal", "食物名称": "food_name"}, inplace=True)


In [89]:
# 创建 unit_weight 列，默认等于原始每份重量
df["unit_weight"] = df["可食部（克/份）"]

# 将可以半份的食物的单位重量减半
df.loc[df["是否可半份"] == 1, "unit_weight"] /= 2


C:\Users\zhangyutang\AppData\Local\Temp\ipykernel_30252\1249068536.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[50.   1.  50.   1.  75.   1.5 50.   1.5 50.   1.5 50.   1.  50.  10.
  1.  75.   1.5 50.   1.5 50.   1.  50.  10.   1.  40.  75.   2.5 75.
 75.   2.5 20.  50.  25.   2.5 75.  50.   2.5 50.   5.   2.5 50.  10.
  2.5 50.  10.   2.5 40.   5.  15.   2.5 50.   7.5  2.5 10.  50.   2.5
 10.  50.   2.5 40.  40.   5.   5.  50.  15.  15.   2.5 50.  15.  15.
  2.5 50.  15.  15.   2.5 40.  15.  15.   2.5 40.   5.  15.  15.   2.5
 75.  15.  15.   2.5 25.  25.   5.   5.   7.5 25.   5.   5.   7.5 25.
 25.   5.  25.  25.   5.  25.   5.   5.   5.   5.  50.   5.   5.   5.
 50.  75.   7.5 50.   5.   5.   5.  50.   1.  75.   1.5 50.   1.5 50.
  1.  50.  10.   1.  25.  50.  50.   2.5 20.  50.  25.   2.5 75.  50.
  2.5 50.   5.   2.5 50.  10.   2.5 50.  10.   2.5 40.   5.  15.   2.5
 50.   7.5  2.5 10.  50. 

In [90]:
# 设置默认上限（不可半份的）
df["x_upper"] = 2

# 可半份的上限翻倍（允许更多“半份”）
df.loc[df["是否可半份"] == 1, "x_upper"] = 4

# 创建整数型决策变量 x_i（每道菜的份数）
df["x_var"] = [
    LpVariable(f"x_{i}", lowBound=0, upBound=df.loc[i, "x_upper"], cat="Integer")
    for i in range(len(df))
]


In [91]:
# 初始化模型
model = LpProblem("Maximize_Total_AAS", LpMaximize)

# 定义三餐AAS评分变量（连续变量，非负）
z_b = LpVariable("z_b", lowBound=0, cat="Continuous")
z_l = LpVariable("z_l", lowBound=0, cat="Continuous")
z_d = LpVariable("z_d", lowBound=0, cat="Continuous")

# 设置目标函数
model += z_b + z_l + z_d, "总AAS评分最大化"


In [92]:
# 氨基酸推荐参考值（单位：mg/g蛋白质）
aas_ref = {
    "异亮氨酸 (mg/100g)": 40,
    "亮氨酸 (mg/100g)": 70,
    "赖氨酸 (mg/100g)": 55,
    "含硫氨基酸(SAA)_Total (mg/100g)": 35,
    "芳香族氨基酸(AAA)_Total (mg/100g)": 60,
    "苏氨酸 (mg/100g)": 40,
    "色氨酸 (mg/100g)": 10,
    "缬氨酸 (mg/100g)": 50,
}


In [94]:
'''
def add_aas_constraints(model, df, z_var, meal_name, P_var):
    """
    为指定餐次添加8种氨基酸的AAS评分线性化约束：
        z_m * R_j * P_m <= AA_j
    参数说明：
        - model：PuLP模型对象
        - df：完整的DataFrame（含x_var、unit_weight、营养素）
        - z_var：本餐的AAS评分变量（如 z_b）
        - meal_name：餐次名（"早餐"、"午餐"、"晚餐"）
        - P_var：该餐蛋白质总量变量（如 P_b）
    """
    df_sub = df[df["meal"] == meal_name]

    # 添加蛋白质总量定义约束（P_m）
    protein_expr = lpSum([
        row["x_var"] * row["unit_weight"] * row["蛋白质 (g/100g)"] / 100
        for _, row in df_sub.iterrows()
    ])
    model += P_var == protein_expr, f"{meal_name}_蛋白质总量定义"

    # 对每个氨基酸添加 z_m * R_j * P_m <= AA_j
    for aa_col, ref_value in aas_ref.items():
        aa_expr = lpSum([
            row["x_var"] * row["unit_weight"] * row[aa_col] / 100
            for _, row in df_sub.iterrows()
        ])
        constraint_name = f"AAS_{meal_name}_{aa_col}"
        model += z_var * ref_value * P_var <= aa_expr, constraint_name
'''

def add_aas_constraints_approx(model, df, z_var, meal_name, P_ref=30):
    """
    为指定餐次添加AAS约束（线性近似版）：
        z_m <= AA_j / (R_j * P_ref)
    其中 P_ref 是一个固定的每餐蛋白质参考值（如 30g）
    """
    df_sub = df[df["meal"] == meal_name]

    for aa_col, ref_value in aas_ref.items():
        aa_expr = lpSum([
            row["x_var"] * row["unit_weight"] * row[aa_col] / 100
            for _, row in df_sub.iterrows()
        ])
        model += z_var <= aa_expr / (ref_value * P_ref)


In [95]:
P_b = LpVariable("P_早餐", lowBound=0)
P_l = LpVariable("P_午餐", lowBound=0)
P_d = LpVariable("P_晚餐", lowBound=0)


In [96]:
add_aas_constraints_approx(model, df, z_b, "早餐")
add_aas_constraints_approx(model, df, z_l, "午餐")
add_aas_constraints_approx(model, df, z_d, "晚餐")


In [99]:
# 计算总能量表达式
E_total = lpSum([
    row["x_var"] * row["unit_weight"] * row["能量 (kcal/100g)"] / 100
    for _, row in df.iterrows()
])

# 添加能量上下限约束
model += E_total >= 2160, "总能量下限1"
model += E_total <= 2640, "总能量上限1"


In [102]:
# 蛋白质能量
E_p = lpSum([
    row["x_var"] * row["unit_weight"] * row["蛋白质 (g/100g)"] / 100 * 4
    for _, row in df.iterrows()
])

# 脂肪能量
E_f = lpSum([
    row["x_var"] * row["unit_weight"] * row["脂肪 (g/100g)"] / 100 * 9
    for _, row in df.iterrows()
])

# 碳水化合物能量
E_c = lpSum([
    row["x_var"] * row["unit_weight"] * row["碳水化合物 (g/100g)"] / 100 * 4
    for _, row in df.iterrows()
])


In [103]:
model += E_p >= 0.10 * E_total, "蛋白质供能下限"
model += E_p <= 0.15 * E_total, "蛋白质供能上限"

model += E_f >= 0.20 * E_total, "脂肪供能下限"
model += E_f <= 0.30 * E_total, "脂肪供能上限"

model += E_c >= 0.50 * E_total, "碳水供能下限"
model += E_c <= 0.65 * E_total, "碳水供能上限"


In [106]:
macro_upper = {
    "能量 (kcal/100g)": 2400,
    "蛋白质 (g/100g)": 90,
    "脂肪 (g/100g)": 80,
    "碳水化合物 (g/100g)": 390,
}

for col, ref in macro_upper.items():
    expr = lpSum([
        row["x_var"] * row["unit_weight"] * row[col] / 100
        for _, row in df.iterrows()
    ])
    model += expr <= 1.1 * ref, f"{col}_摄入上限_110%1"


In [109]:
# 钙
model += lpSum([
    row["x_var"] * row["unit_weight"] * row["钙 (mg/100g)"] / 100
    for _, row in df.iterrows()
]) >= 800, "钙摄入下限1"

# 铁
model += lpSum([
    row["x_var"] * row["unit_weight"] * row["铁 (mg/100g)"] / 100
    for _, row in df.iterrows()
]) >= 12, "铁摄入下限1"

# 锌
model += lpSum([
    row["x_var"] * row["unit_weight"] * row["锌 (mg/100g)"] / 100
    for _, row in df.iterrows()
]) >= 12.5, "锌摄入下限1"

# 维生素A（μg）
model += lpSum([
    row["x_var"] * row["unit_weight"] * row["总维生素A (μg/100g)"] / 100
    for _, row in df.iterrows()
]) >= 800, "维A摄入下限1"

# 维生素B1（硫胺素），字段单位为 μg，要转为 mg
model += lpSum([
    row["x_var"] * row["unit_weight"] * row["硫胺素 (μg/100g)"] / 100 / 1000
    for _, row in df.iterrows()
]) >= 1.4, "VB1摄入下限1"

# 维生素B2（核黄素）
model += lpSum([
    row["x_var"] * row["unit_weight"] * row["核黄素 (mg/100g)"] / 100
    for _, row in df.iterrows()
]) >= 1.4, "VB2摄入下限1"

# 维生素C
model += lpSum([
    row["x_var"] * row["unit_weight"] * row["维生素C (mg/100g)"] / 100
    for _, row in df.iterrows()
]) >= 100, "VC摄入下限1"


In [110]:
# 添加 0/1辅助变量 y_i
df["y_var"] = [LpVariable(f"y_{i}", cat="Binary") for i in range(len(df))]


In [111]:
for _, row in df.iterrows():
    model += row["x_var"] <= row["x_upper"] * row["y_var"]


In [112]:
model += lpSum(df["y_var"]) >= 12, "每日食物种类数不少于12"
for meal in ["早餐", "午餐", "晚餐"]:
    model += lpSum(df[df["meal"] == meal]["y_var"]) <= 7, f"{meal}_食物种类数不超过7"


In [113]:
# 推荐摄入上限（110%）设置
nutrient_limits = {
    "钙 (mg/100g)": 800,
    "铁 (mg/100g)": 12,
    "锌 (mg/100g)": 12.5,
    "总维生素A (μg/100g)": 800,
    "硫胺素 (μg/100g)": 1400,   # 单位是μg，推荐值1.4mg
    "核黄素 (mg/100g)": 1.4,
    "维生素C (mg/100g)": 100,
}

for col, ref in nutrient_limits.items():
    # B1需要特殊处理（单位是μg，推荐值是mg）
    factor = 1 / 1000 if col == "硫胺素 (μg/100g)" else 1
    upper_expr = lpSum([
        row["x_var"] * row["unit_weight"] * row[col] / 100 * factor
        for _, row in df.iterrows()
    ])
    model += upper_expr <= 1.1 * (ref * factor), f"{col}_摄入上限_110%"


In [115]:
model.solve()  # 也可用 msg=0 静默
print(f"模型求解状态：{LpStatus[model.status]}")

# 保存最优解
df["x_opt"] = df["x_var"].apply(value)
df["y_opt"] = df["y_var"].apply(value)

for meal in ["早餐", "午餐", "晚餐"]:
    print(f"\n🍽 {meal} 推荐菜品：")
    df_meal = df[(df["meal"] == meal) & (df["x_opt"] > 0)]
    for _, row in df_meal.iterrows():
        grams = row["unit_weight"] * row["x_opt"]
        print(f" - {row['food_name']}：{row['x_opt']} 份（约 {grams:.1f} 克）")

print(f"\n🥩 AAS评分：")
print(f" - 早餐 AAS：{value(z_b):.4f}")
print(f" - 午餐 AAS：{value(z_l):.4f}")
print(f" - 晚餐 AAS：{value(z_d):.4f}")
print(f" - 总AAS得分：{value(z_b + z_l + z_d):.4f}")

# 各营养素单位：g/100g，能量 kcal/100g
df["E_kcal"] = df["x_opt"] * df["unit_weight"] * df["能量 (kcal/100g)"] / 100
df["P_g"] = df["x_opt"] * df["unit_weight"] * df["蛋白质 (g/100g)"] / 100
df["F_g"] = df["x_opt"] * df["unit_weight"] * df["脂肪 (g/100g)"] / 100
df["C_g"] = df["x_opt"] * df["unit_weight"] * df["碳水化合物 (g/100g)"] / 100

E_total = df["E_kcal"].sum()
E_P = df["P_g"].sum() * 4
E_F = df["F_g"].sum() * 9
E_C = df["C_g"].sum() * 4

print(f"\n🔥 总能量摄入：{E_total:.1f} kcal")
print(f" - 蛋白质占比：{E_P / E_total:.2%}")
print(f" - 脂肪占比：{E_F / E_total:.2%}")
print(f" - 碳水占比：{E_C / E_total:.2%}")


模型求解状态：Infeasible

🍽 早餐 推荐菜品：
 - 牛奶：2.0 份（约 400.0 克）
 - 酸奶：2.0 份（约 250.0 克）
 - 豆浆：2.0 份（约 20.0 克）
 - 大米粥：2.0 份（约 30.0 克）
 - 小米粥：2.0 份（约 30.0 克）
 - 大米饭：2.0 份（约 50.0 克）
 - 馒头：2.0 份（约 100.0 克）
 - 花卷：2.0 份（约 100.0 克）
 - 油条：2.0 份（约 100.0 克）
 - 油条：2.0 份（约 20.0 克）
 - 煮鸡蛋：2.0 份（约 100.0 克）
 - 煎鸡蛋：2.0 份（约 100.0 克）
 - 煎鸡蛋：2.0 份（约 20.0 克）
 - 蒸地瓜：2.0 份（约 200.0 克）
 - 南瓜粥：2.0 份（约 20.0 克）
 - 南瓜粥：2.0 份（约 40.0 克）
 - 馄饨：2.0 份（约 80.0 克）
 - 馄饨：2.0 份（约 50.0 克）
 - 馄饨：2.0 份（约 20.0 克）
 - 鸡排面：2.0 份（约 100.0 克）
 - 鸡排面：2.0 份（约 80.0 克）
 - 鸡排面：2.0 份（约 10.0 克）
 - 馄饨面：2.0 份（约 100.0 克）
 - 馄饨面：2.0 份（约 30.0 克）
 - 馄饨面：2.0 份（约 80.0 克）
 - 馄饨面：2.0 份（约 80.0 克）
 - 馄饨面：2.0 份（约 20.0 克）
 - 包子：2.0 份（约 50.0 克）
 - 包子：2.0 份（约 30.0 克）
 - 包子：2.0 份（约 30.0 克）
 - 包子：2.0 份（约 60.0 克）
 - 包子：2.0 份（约 10.0 克）
 - 馅饼：2.0 份（约 50.0 克）
 - 馅饼：2.0 份（约 20.0 克）
 - 馅饼：2.0 份（约 20.0 克）
 - 馅饼：2.0 份（约 20.0 克）
 - 鸡蛋饼：2.0 份（约 50.0 克）
 - 鸡蛋饼：2.0 份（约 40.0 克）
 - 鸡蛋饼：2.0 份（约 40.0 克）
 - 鸡蛋饼：2.0 份（约 10.0 克）
 - 土豆丝饼：2.0 份（约 40.0 克）
 - 土豆丝饼：2.0 份（约 100.0 克）
 - 土豆丝饼：2.